In [2]:
from Bio import Entrez
Entrez.email = "wuchangsheng@sdu.edu.cn"
import os
import pickle
import json
import lzma
from datetime import date
from time import strftime, sleep
from urllib.error import HTTPError
from http.client import IncompleteRead
import sys
from Bio import SeqIO
from Bio import Seq
import subprocess

# Set these 3 parameter
prefix = "txid29"
searchTerm = f'{prefix}[Organism] AND refseq[filter] NOT "sequencing project"[Title]'
targetDir = "/mnt/sdb/wcs/NCBI-myxobacterial-genome"
######

searchStatusFile = os.path.join(targetDir, 'searchStatus.json')
if os.path.isfile(searchStatusFile):
    with open(searchStatusFile,'r') as h:
        newSearch, dateOfSearch, start, downloadStep = json.load(h)
else:
    newSearch = True
    start = 0
    dateOfSearch = date.today().strftime("%Y%m%d")
    downloadStep = 1 # this may change only when you start a new search


def writeLog(logFile, logStr, end = '\n'):
    print(logStr, end = end)
    with open(logFile,'a') as logHandle:
        logHandle.write(f'{logStr}{end}')

* Taxonomy ID: 228398 (for references in articles please use NCBI:txid228398)  
 Scientific name: Streptacidiphilus Kim et al. 2003
* Taxonomy ID: 2063 (for references in articles please use NCBI:{prefix})  
 Scientific name: Kitasatospora corrig. Omura et al. 1983 emend. Zhang et al. 1997
* Taxonomy ID: 1883 (for references in articles please use NCBI:txid1883)  
 Scientific name: Streptomyces Waksman and Henrici 1943 (Approved Lists 1980) emend. Wellington et al. 199* 
* Taxonomy ID: 1914443 (for references in articles please use NCBI:txid1914443)  
 Scientific name: Allostreptomyces Huang et al. 2017
* Taxonomy ID: 329648 (for references in articles please use NCBI:txid329648)  
 Scientific name: "Parastreptomyces" 1) Nichols et al. 2005
* Taxonomy ID: 65518 (for references in articles please use NCBI:txid65518)  
 Scientific name: "Trichotomospora" 1) Lian et al. 1985
* Taxonomy ID: 234292 (for references in articles please use NCBI:txid234292)  
 Scientific name: unclassified Streptomycetaceae
* Taxonomy ID: 259296 (for references in articles please use NCBI:txid259296)  
 Scientific name: environmental samples

In [ ]:
def getRecords(searchTerm, c = 0, step = 2000, webaccession = None, targets = []):
    count = 0
    while c < count or count == 0:
        print(f'No.{int(c/step+1)}', end = '>')
        handle = Entrez.esearch(db='nuccore', 
                                term=searchTerm,
                                usehistory=True,
                                webenv=webaccession, # reuse teh first query
                                retstart=c, # Continue from num c
                                retmax=step # Maxium number returned
                               )
        record = Entrez.read(handle)
        handle.close()
        if record['Count'] == "0":
            print("\nThere is no genome with this search.")
            break
        if 'WarningList' in record: # If anything wrong
            print(record['WarningList']['OutputMessage'])

        # Put current list of IDs in our targets dict    
        targets.append(record['IdList'])
        print(f"{len(record['IdList'])}", end = '|')  # the number got from this loop.

        if webaccession == None:
            webaccession = record["WebEnv"] # start new search only in the first loop
            print(f'\nWebAccession:\n{webaccession}')
            count = int(record['Count']) # total number of hits get from the first search attempt
            print(f"Total entry {count}.")    
        c += step
    return c, count, webaccession, targets

In [ ]:
if newSearch:
    if "webaccession" in globals():
        c, count, webaccession, targets = getRecords(searchTerm=searchTerm, c=c, step=200, webaccession=webaccession, targets=targets)
    else:
        c, count, webaccession, targets = getRecords(searchTerm = searchTerm, step=200)

    # now print and check the first 5 IDs of the first 10 lists of IDs
    print(f"\nTotal target sets {len(targets)}\nFirst 10:")
    for targ in targets[:10]:
        print(f"{targ[:5]}...")
    print('...')
else:
    print('Skip')


# Fetching all data from online (getting IDs from previous stored data)

In [ ]:
# Dump targets got from search in pickle file
targetsJson = os.path.join(targetDir, f'refseq_{prefix}_{dateOfSearch}.json')
print(targetsJson)
print(newSearch)

if newSearch:
    with open(targetsJson, 'w') as to:
        json.dump(targets, to)
    newSearch = False # switch off newSearch
else:
    with open(targetsJson, 'r') as ti:
        targets = json.load(ti)
        
# flatten the targets list of list
# remove redundant if we have somene


nucids = sorted(list(set(item for sublist in targets for item in sublist)))
print(f"Total nucids {len(nucids)}\nRemember to switch off <newSearch>")

In [ ]:
from multiprocessing import Process, Queue

def fetch(succeed, index, file_Nu, ids, returnType = ['fasta','fasta']):
    
    '''fetch(index, file_Nu, ids, returnType = ['fasta','fasta'])
    
    other option is:
    returnType = ['gbwithparts','gb']'''

    writeLog(logFile, f"Fetching {file_Nu}: {ids[0]}...({len(ids)})")
    
    output_file = os.path.join(outputPath, f'{prefix}_No_{file_Nu}.{returnType[1]}')

    # Fetching...
    with Entrez.efetch(db = 'nuccore',
                       id = ids,
                       rettype = returnType[0],
                       retmode = 'text'
                      ) as handle:
        with open(output_file, 'w') as out_handle:
            out_handle.write(handle.read())
    
    # Write finishing note to screen and log file
    logstr = f"Finished {file_Nu}: {os.stat(output_file).st_size/1024/1024:.2f} MB {output_file} \n"
    print(logstr)
    with open(logFile,'a') as log_handle:
        log_handle.write(f'{logstr}\n')
    #return True
    succeed.put(True)


def finishing(allDone = True):
    totalSize = 0 # calculate total amount data got from entrez
    for file in os.listdir(outputPath):
        if file.endswith('.gb') or file.endswith('.fasta'):
            totalSize += os.stat(os.path.join(outputPath,file)).st_size
    
    writeLog(logFile, f"Group finished, already got {totalSize/1024/1024:.2f} MB data!")
    if allDone:
            writeLog(logFile,'Finished fetching all IDs.')
    else:
        writeLog(logFile, f'Next download will start from group {start} (of 0 - {len(idStacks)-1}).')
        writeLog(logFile, f'{timestamp:*^80}')
        print('Before starting next query, please set how many groups you want to fetch based on your schedule.')        

def fetchData(start, idStacks, getSingleBatch = False):
    for i in range(start, len(idStacks)):
        # Break the loop for shorter operation and debugging time
        if i == end and i != len(idStacks)-1:
            start = i
            finishing(allDone = False)
            return True, start

        ids = idStacks[i]

        succeed = False
        retryTimes = 0
        while succeed == False:
            try:
                processSucceed = Queue()
                fetch_process = Process(target = fetch, kwargs={'index':i,
                                                                'succeed':processSucceed,
                                                                'file_Nu':str(i).zfill(5),
                                                                'ids':ids,
                                                                'returnType':['gbwithparts','gb']
                                                               })
                fetch_process.start()
                fetch_process.join(timeout=60*60)
                if processSucceed.empty():
                    raise Exception
                else:
                    succeed = processSucceed.get()
                #succeed = fetch(index = i, file_Nu = str(i).zfill(4), ids = ids, returnType = ['gbwithparts','gb'])
            except (HTTPError, IncompleteRead):
                retryTimes += 1
                if retryTimes == 10:
                    writeLog(logFile,"Failed 10 times due to HTTPError (NCBI server problem), please try again another time.")
                    start = i
                    writeLog(logFile,f'Start value set for next trial. [{start}]')
                    return False, start
                else:
                    writeLog(logFile,'Failed due to HTTPError, pause for 45 seconds...',end = '')
                    sleep(45)
                    writeLog(logFile,'Retrying...')
            except Exception as e:
                start = i
                writeLog(logFile,f'Failed due to uncought error\n{e}\nStart value set for next trial. [{start}]')

                return False, start
            finally:
                with open(searchStatusFile,'w') as h:
                    json.dump([False, dateOfSearch, start, downloadStep], h)
        if not succeed:
            writeLog(logFile,"I don't know what happend, ask DU")
            return False, start
        elif i == len(idStacks) - 1:
            finishing(allDone = True)
            return True, 0
        if getSingleBatch:
            break
            return True, 0


In [ ]:
# For debugging
start = 1271 # should start from 0 if nothing have downloaded
downloadStep=1
# getSingleBatch = False

In [7]:
# define how many sequences to download in each group
idStacks = []
for i in range(0,len(nucids),downloadStep):
    idStacks.append(nucids[i:i+downloadStep])

print(f'Last group No. {str(len(idStacks)-1).zfill(5)}')

numToFetch = 99999 # number of files to fetch (specifing if you can not finish in one go)

end = start + numToFetch # range(start, end) means not including end number!!

if end > len(idStacks):
    end = len(idStacks)

outputPath = r"/mnt/sdb/wcs/NCBI-myxobacterial-genome"
logFile = os.path.join(outputPath, f'fetching.log')
timestamp = strftime('%X %d/%m/%Y %Z')

writeLog(logFile, f'\n{timestamp:*^80}\nNow fetching groups from {start} to {end-1}\nEach group have {downloadStep} nuclIDs\nIDs from {start*downloadStep+1} to {end*downloadStep+1-1} (inclusive)\n{"":*^80}\n')
# Mean loop
isDone, start = fetchData(start,idStacks)
while not isDone:
    isDone, start = fetchData(start,idStacks)
    sleep(15)

Finished 23273: 0.01 MB /mnt/sdb/wcs/NCBI-myxobacterial-genome/txid29_No_23273.gb 

Fetching 23274: 1697872765...(1)
Finished 23274: 0.01 MB /mnt/sdb/wcs/NCBI-myxobacterial-genome/txid29_No_23274.gb 

Fetching 23275: 1697872767...(1)
Finished 23275: 0.01 MB /mnt/sdb/wcs/NCBI-myxobacterial-genome/txid29_No_23275.gb 

Fetching 23276: 1697872770...(1)
Finished 23276: 0.00 MB /mnt/sdb/wcs/NCBI-myxobacterial-genome/txid29_No_23276.gb 

Fetching 23277: 1697872772...(1)
Finished 23277: 0.01 MB /mnt/sdb/wcs/NCBI-myxobacterial-genome/txid29_No_23277.gb 

Fetching 23278: 1697872774...(1)
Finished 23278: 0.01 MB /mnt/sdb/wcs/NCBI-myxobacterial-genome/txid29_No_23278.gb 

Fetching 23279: 1697872776...(1)
Finished 23279: 0.00 MB /mnt/sdb/wcs/NCBI-myxobacterial-genome/txid29_No_23279.gb 

Fetching 23280: 1697872778...(1)
Finished 23280: 0.01 MB /mnt/sdb/wcs/NCBI-myxobacterial-genome/txid29_No_23280.gb 

Fetching 23281: 1698391930...(1)
Finished 23281: 0.59 MB /mnt/sdb/wcs/NCBI-myxobacterial-genome/t

Process Process-17194:
Process Process-18468:
Process Process-18353:
Process Process-17120:
Process Process-17133:
Process Process-1:
Process Process-18454:
Process Process-17568:
Process Process-18479:
Traceback (most recent call last):
  File "/mnt/sdb/wcs/.conda/envs/jupyter-install/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/mnt/sdb/wcs/.conda/envs/jupyter-install/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "<ipython-input-5-be08f5423696>", line 21, in fetch
    out_handle.write(handle.read())
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/mnt/sdb/wcs/.conda/envs/jupyter-install/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Trace

# Convert genbank file to blast database

gb files needs to be converet to fasta file before making a database  
also the files needed to be combined incase combine blast database don't work, also for easy file transfering

In [ ]:
def convertGb2Fasta(storage_folder_gb, storage_folder_fa, logFile, sizeLimit, numFilesToConvert = 99999):
    print('Changing gb to fasta...')
    
    totalGbs = [int(file.split('.')[0].split('_')[-1]) for file in os.listdir(storage_folder_gb) if file.endswith('gb')]
    totalGbs.sort()

    
    # write log file
    writeLog(logFile, f'\n{timestamp:*^50}\n{totalGbs[-1]} of gb files in total\n{"":*^50}\n')
    # setup starting values
    recordWithSeq = [] # container for seqs for the function of dumping certain size of seqs together.
    seqTotalLength = 0
    num_empty = 0
    breakPoint = 0
    
    # Main loop
    for i in totalGbs:
        num = str(i).zfill(5)
        input_file = os.path.join(storage_folder_gb, f'{prefix}_No_{num}.gb') 
        # If there is a gap in the input file, it will currupt here.

        # Write note to screen and log file
        if i%5 ==0:
            writeLog(logFile, f"{num}|{seqTotalLength}", end = '\t')
            if i%25 ==0:
                writeLog(logFile, '')
            
        records = SeqIO.parse(input_file, 'genbank')
        for record in records:
            if type(record.seq) == Seq.UnknownSeq: # Empty records will load as UnknownSeq
                num_empty += 1
                pass
            else:
                recordWithSeq.append(record)
                seqTotalLength += len(record)
        if seqTotalLength >= sizeLimit:
            writeLog(logFile, '\nReach the size limit of single file, writing it out')
            output_file = os.path.join(storage_folder_fa, f'{prefix}_No_{breakPoint}to{num}.fasta')
            with open(output_file, 'w') as fasta_out_handle:
                if len(recordWithSeq) == 0: # if all records are empty, there is no point of writing it to fasta
                    writeLog(logFile, f'There is no sequence in this batch, proceed to next file...')
                else:
                    SeqIO.write(recordWithSeq, fasta_out_handle, 'fasta')
                    writeLog(logFile, f"Finished converting {num}, ignored {num_empty} empty records in this batch.")
            # Reset batch
            recordWithSeq = list()
            seqTotalLength = 0
            num_empty = 0
            breakPoint = str(i).zfill(5)
    writeLog(logFile, '\nReach the end, writing it out...')
    output_file = os.path.join(storage_folder_fa, f'{prefix}_No_{breakPoint}to{num}.fasta')
    with open(output_file, 'w') as fasta_out_handle:
        if len(recordWithSeq) == 0: # if all records are empty, there is no point of writing it to fasta
            writeLog(logFile, f'There is no sequence in this last batch.')
        else:
            SeqIO.write(recordWithSeq, fasta_out_handle, 'fasta')
            writeLog(logFile, f"Finished converting {num}, ignored {num_empty} empty records in this batch.")

            
            
timestamp = strftime('%X %d/%m/%Y %Z')
fastaFolder = '/mnt/sdb/wcs/NCBI-myxobacterial-genome/fasta'
fetched = '/mnt/sdb/wcs/NCBI-myxobacterial-genome'
if not os.path.isdir(fastaFolder):
    os.mkdir(fastaFolder)
logFile = os.path.join(fastaFolder,'converting.log')
print('Converting...')
sizeLimit = 400000000 # if the file is bigger than 400 M, program will write it out and start a new file

convertGb2Fasta(fetched,fastaFolder,logFile,sizeLimit)


In [ ]:
def combineGenbank(fetchedGenbank,combinedGenbank,logFile,sizeLimit,startFileNum = 0):
    if not os.path.isdir(combinedGenbank):
        os.mkdir(combinedGenbank)

    fileList = [os.path.join(fetchedGenbank, file) for file in os.listdir(fetchedGenbank) if file.endswith('.gb')]
    seqTotalLength = 0
    seqsInOneFile = []
    breakPoint = startFileNum
    for i in range(startFileNum, len(fileList)):
        parseSeqs = list(SeqIO.parse(fileList[i],'genbank'))
        for seq in parseSeqs:
            seqTotalLength += len(seq)
            if len(seq) == 0:
                print('There is 0 length sequences')
                break
        seqsInOneFile += parseSeqs
        logStr = f"{i}|{seqTotalLength}"
        writeLog(logFile, logStr, end = '\t')

        if i == len(fileList) - 1:
            outputFile = os.path.join(combinedGenbank, f'{prefix}_{breakPoint}_to_{i}.gb')
            logStr = f'\nFinished combining, write out last batch...\n{outputFile}'
            writeLog(logFile, logStr)
            SeqIO.write(seqsInOneFile, outputFile, 'genbank')
            writeLog(logFile, 'Write succeeded.')
        elif seqTotalLength >= sizeLimit:
            outputFile = os.path.join(combinedGenbank, f'{prefix}_{breakPoint}_to_{i}.gb')
            logStr = f'\nReached file size limit, write out...\n{outputFile}'
            writeLog(logFile, logStr)
            SeqIO.write(seqsInOneFile, outputFile, 'genbank')
            breakPoint = i+1
            seqsInOneFile = []
            seqTotalLength = 0
            writeLog(logFile, 'Write succeeded.')

            
timestamp = strftime('%X %d/%m/%Y %Z')
combinedFolder = r"/mnt/sdb/wcs/NCBI-myxobacterial-genome"
sourceFolder = r"/mnt/sdb/wcs/NCBI-myxobacterial-genome"
logFile = os.path.join(combinedFolder,'converting.log')
print('Converting...')
sizeLimit = 1000000000

# 1000000000 cost about 4-6 GB memory for genbank files, result ~2.5 GB file.
startFileNum = 624
combineGenbank(sourceFolder,combinedFolder,logFile,sizeLimit,startFileNum)


# Make blast database

In [7]:
sourceDir = '/mnt/sdb/wcs/NCBI-myxobacterial-genome/fasta'
outputDir = '/mnt/sdb/wcs/NCBI-myxobacterial-genome/db'
logFile = '/mnt/sdb/wcs/NCBI-myxobacterial-genome/db/mkdb.log'

if not os.path.isdir(outputDir):
    os.mkdir(outputDir)
    
# Change here accroding to the format fetched from entrez 
totalNum = sum(file.endswith('fasta') for file in os.listdir(sourceDir))
print(f'Total number of files to be converted {totalNum}.\nConverted:', end='')

converted = 0
for file in os.listdir(sourceDir):
    if not file.endswith('fasta'):
        continue
    singleSeqFile = os.path.join(sourceDir, file)
    args = ['makeblastdb',
            '-in', singleSeqFile,
            '-input_type', 'fasta',
            '-dbtype', 'nucl',
            '-title', f'{file[:-6]}',
            '-out', os.path.join(outputDir, f'{file[:-6]}_nucl'),
            '-logfile', logFile,
            '-taxid', '29'
           ]
    run = subprocess.run(args, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    if run.returncode == 0:
        converted += 1
        print(f'{converted}|', end = '')
        pass
    else:
        with open(logFile, 'r') as log:
            for line in log.readlines():
                print(line)
        break
print(f'\n\nFinished, databases made: {converted}')

Total number of files to be converted 3.
Converted:1|2|3|

Finished, databases made: 3


## Merge blast database into one

In [8]:
databaseDir = '/mnt/sdb/wcs/NCBI-myxobacterial-genome/db'
listFile = '/mnt/sdb/wcs/NCBI-myxobacterial-genome/db/listofdbs.txt'
databaseList = []
for file in os.listdir(databaseDir):
    if not file.endswith('nsq'):
        continue
    dbName = file.split('.')[0]
    databaseList.append(os.path.join(databaseDir, dbName))
with open(listFile, 'w') as handle:
    handle.write('\n'.join(databaseList))
    
args = ['blastdb_aliastool',
        '-dblist_file', listFile,
        '-dbtype', 'nucl',
        '-out', os.path.join(databaseDir, f'{prefix}Nucl{dateOfSearch}'),
        '-title', f'{prefix}Nucl{dateOfSearch}',
       ]
run = subprocess.run(args, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
if run.returncode == 0:
    pass
else:
    print(run.stdout.decode())
    print(run.stderr.decode())